In [2]:
import pandas as pd
import json
import numpy as np
import plotly.express as px
from urllib.request import urlopen

#GeoJSON stadsdelen Amsterdam
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
with urlopen(GEOJSON_URL) as response:
    stadsdelen = json.load(response)

#Get Sensors from data
df = pd.read_csv("sensor-location.xlsx - Sheet1.csv")
df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
df["Lat"]  = pd.to_numeric(df["Lat"],  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"], errors="coerce")

#Some sensors don't have "Breedte", so it gives "Effectieve breedte"
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

#RANDOM data to test for heatmap 
rng = np.random.default_rng(42)
df["Persons"] = rng.integers(0, 200, size=len(df))

#HEATMAP
fig = px.density_mapbox(
    df,
    lat="Lat",
    lon="Long",
    z="Persons",                     # gewicht: meer persons = hogere intensiteit
    radius=60,                       # straal in meters (speel hiermee!)
    center={"lat": 52.372, "lon": 4.900},
    zoom=10,
    color_continuous_scale="YlOrRd", # geel→rood
    opacity=0.7,
    height=700
)

fig.update_layout(
    mapbox_style="carto-positron",   # of "open-street-map" als je geen token hebt
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()


C:\Users\matsj\AppData\Local\Temp\ipykernel_12672\2761830273.py:28: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [5]:
# ==========================================
# Crowd heatmap monitor (Plotly Express)
# - leest cs: predicted_sensor_values_3min.csv
# - koppelt automatisch sensoren aan locaties
# - animeert 1 frame per timestamp (~1s)
# ==========================================

import pandas as pd
import json
import numpy as np
import plotly.express as px
from urllib.request import urlopen

# ---------- 0) Paden ----------
CS_CSV = r"predicted_sensor_values_3min.csv"          # crowd sensor waarden (wide format)
SENSOR_META_CSV = r"sensor-location.xlsx - Sheet1.csv" # sensor locaties (Lat/Long + sensor-id kolom)

# ---------- 1) (Optioneel) stadsdelen ----------
GEOJSON_URL = "https://maps.amsterdam.nl/open_geodata/geojson_lnglat.php?KAARTLAAG=INDELING_STADSDEEL&THEMA=gebiedsindeling"
try:
    with urlopen(GEOJSON_URL) as response:
        stadsdelen = json.load(response)
except Exception:
    stadsdelen = None  # niet kritisch voor de heatmap

# ---------- 2) Sensorlocaties inladen ----------
df = pd.read_csv(SENSOR_META_CSV)

# Verwacht een kolom "Lat/Long" als "lat,lon" string
if "Lat/Long" in df.columns:
    df[["Lat", "Long"]] = df["Lat/Long"].str.split(",", expand=True)
else:
    # Als je al aparte kolommen hebt, hernoem ze evt. hier:
    # df = df.rename(columns={"latitude_col": "Lat", "longitude_col": "Long"})
    pass

df["Lat"]  = pd.to_numeric(df["Lat"],  errors="coerce")
df["Long"] = pd.to_numeric(df["Long"], errors="coerce")

# Klein schoonmaakje: niet nodig voor plot, maar stoort ook niet
for col in ["Breedte", "Effectieve breedte"]:
    if col in df.columns:
        df[col] = df[col].fillna("Effectieve breedte")

# ---------- 3) Crowddata (cs) inladen ----------
# Expected: kolommen = tijdsvelden + sensor-kolommen
# timestamp-kolom heet 'timestamp' (string of datetime)
cs = pd.read_csv(CS_CSV)

# Welke tijdsvelden zijn er echt aanwezig?
possible_time_cols = ["timestamp","hour","minute","day","month","weekday","is_weekend"]
time_cols = [c for c in possible_time_cols if c in cs.columns]
if "timestamp" not in time_cols:
    raise ValueError("Kolom 'timestamp' ontbreekt in je CSV. Voeg deze toe of pas de code aan.")

# Zet timestamp naar datetime voor sorteren/animatie
cs["timestamp"] = pd.to_datetime(cs["timestamp"], errors="coerce")

# Sensor-kolommen = alles behalve tijdsvelden
sensor_cols = [c for c in cs.columns if c not in time_cols]

if not sensor_cols:
    raise ValueError("Geen sensor-kolommen gevonden in cs. Controleer je CSV.")

# ---------- 4) Probeer automatisch de sensor-ID kolom in df te vinden ----------
# We kiezen de string-kolom in df met de grootste overlap met sensor_cols
sensor_set = set(sensor_cols)
candidate_cols = [c for c in df.columns if df[c].dtype == object]

best_col = None
best_overlap = 0.0
for c in candidate_cols:
    values = set(df[c].dropna().astype(str))
    overlap = len(values & sensor_set) / max(1, len(sensor_set))
    if overlap > best_overlap:
        best_overlap = overlap
        best_col = c

# Als overlap mager is, val terug op expliciete naam of op volgorde
if best_overlap >= 0.2:
    SENSOR_COL = best_col
else:
    # Probeer veel voorkomende naam
    SENSOR_COL = "Sensor" if "Sensor" in df.columns else None

# ---------- 5) Crowddata smelten naar long ----------
cs_long = cs.melt(
    id_vars=time_cols,
    value_vars=sensor_cols,
    var_name="sensor",
    value_name="value"
)

# ---------- 6) Merge waarden met locaties ----------
if SENSOR_COL is not None and SENSOR_COL in df.columns:
    merged = cs_long.merge(
        df[[SENSOR_COL, "Lat", "Long"]],
        left_on="sensor",
        right_on=SENSOR_COL,
        how="left"
    )
else:
    # Fallback: orden sensoren alfabetisch en mappen op eerste N rijen van df
    df_tmp = df.copy().reset_index(drop=True)
    df_tmp = df_tmp.loc[:len(sensor_cols)-1].copy()
    df_tmp["sensor"] = sorted(sensor_cols)
    merged = cs_long.merge(
        df_tmp[["sensor", "Lat", "Long"]],
        on="sensor",
        how="left"
    )

# Verwijder rijen zonder locatie
merged = merged.dropna(subset=["Lat","Long"]).copy()

# ---------- 7) Sorteren & animatie-as ----------
merged = merged.sort_values(["timestamp", "sensor"])
merged["timestamp_str"] = merged["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")

# ---------- 8) Plotly density heatmap met animatie ----------
fig = px.density_mapbox(
    merged,
    lat="Lat",
    lon="Long",
    z="value",                       # intensiteit: aantal personen
    radius=60,                       # straal in meters
    center={"lat": 52.372, "lon": 4.900},
    zoom=10,
    color_continuous_scale="YlOrRd",
    opacity=0.7,
    height=720,
    animation_frame="timestamp_str"  # 1 frame per timestamp
)

fig.update_layout(
    mapbox_style="carto-positron",   # of "open-street-map"
    margin={"r":0,"t":30,"l":0,"b":0},
    title="Crowd heatmap – animatie per timestamp"
)

# 1 seconde per frame
if fig.layout.updatemenus and fig.layout.updatemenus[0].buttons:
    fig.layout.updatemenus[0].buttons[0].args[1]["frame"] = {"duration": 1000, "redraw": False}
    fig.layout.updatemenus[0].buttons[0].args[1]["transition"] = {"duration": 0}

fig.show()

#---------- 9) (Optioneel) sanity checks ----------
print("Gevonden sensor-id kolom in df:", SENSOR_COL if SENSOR_COL else "fallback (alfabetische mapping)")
print("Aantal frames (timestamps):", merged["timestamp_str"].nunique())
print("Aantal unieke sensoren:", merged["sensor"].nunique())


C:\Users\matsj\AppData\Local\Temp\ipykernel_12672\998446222.py:121: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



Gevonden sensor-id kolom in df: fallback (alfabetische mapping)
Aantal frames (timestamps): 30
Aantal unieke sensoren: 24
